- Download chromedriver for the version of google chrome on your device.

Download chromedriver for the version of google chrome on your device. 
https://developer.chrome.com/docs/chromedriver/downloads
Note the warning at the top of the page: if you are using Chrome version 115 or newer, consult the Chrome for Testing availability dashboard. 


incase you get this error, you can check your path to ensure you had not installed a prevoius version of the chrome driver
'''The chromedriver version (126.0.6478.182) detected in PATH at C:\Program Files\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (128.0.6613.137); currently, chromedriver 128.0.6613.137 is recommended for chrome 128.*, so it is advised to delete the driver in PATH and retry'''





- Save it in your environmental virable or in a location you like.

- create a new virtualenv 
python -m venv venv





In [10]:
# ! pip install selenium
# ! pip install selenium-stealth
# ! pip install beautifulsoup4
# ! pip install lxml
# ! pip install pandas

In [1]:
from selenium import webdriver
from selenium_stealth import stealth

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import requests
import csv
import time
from bs4 import BeautifulSoup

In [2]:
# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run headless Chrome
chrome_options.add_argument("--window-size=1920,1080")  # Set window size
# chrome_options.add_argument('--headless=new') # ensure GUI is off
chrome_options.add_argument('start-maximized') #sets the browser to maximixed view
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-search-engine-choice-screen")
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--window-size=2560,1440") # set specific window size for the browser,
chrome_options.add_argument("--incognito") # set the browser mode to incognito
chrome_options.add_argument('--enable-javascript')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"]) # selenium stealth option to enable automation
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument("lang=en")
chrome_options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=chrome_options)

#if you did not set the chrome driver to you environmental variable, uncomment the line below.
# driver = webdriver.Chrome(options=chrome_options, executable_path=r"path_to_extracted_driver\chromedriver.exe")




In [3]:
url = 'https://www.kayak.com/flights/ROM-MAD/2024-10-03?fs=cfc=1&sort=bestflight_a'

In [4]:
driver.get(url=url)
driver.implicitly_wait(5)

#handle cookie pop up
try:
    # Wait for the cookie popup and click 'Accept all' or 'Reject all'
    cookie_popup = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.XPATH, "//div[text()='Accept all']"))).click()

    
except TimeoutException:
    print("Cookie consent popup not found")



more_buttons = driver.find_element(By.CLASS_NAME,'show-more-button').click() #Find the show more button at the bottom of the page

# Use the WebDriver implicit wait function to make the page wait for a while (max the number of seconds specified) until it can find the button.
# This wait is added to ensure that the show more button is loaded and clicked.
# During the wait, the line below also indicates that the webdeiver should ignore some exceptions.
# When the show more button is visible then the web browser showld click on it.
WebDriverWait(driver,5,
              ignored_exceptions=(
                      NoSuchElementException, 
                      StaleElementReferenceException)).until(EC.visibility_of_element_located((By.CLASS_NAME,'show-more-button'))).click()

# if you want to click the button multiple times then you can use a loop. in the loop, 
# I used an explicit wait of 2 sec to force the page to wait for the readmore button. this is to show other ways of waiting.
# The explicit wait will wait until the time specified is over.
for _ in range(2):

    try:
            time.sleep(2)
            WebDriverWait(driver,5,ignored_exceptions=( NoSuchElementException, StaleElementReferenceException,ElementClickInterceptedException)).until(EC.visibility_of_element_located((By.CLASS_NAME,'show-more-button'))).click()
    except TimeoutException:
            break


In [5]:
# Click on the card of each flight so it shows other information

pop_out_buttons = driver.find_elements(By.CLASS_NAME,"nrc6-content-section")
for x in range(len(pop_out_buttons)):
    try:
        if pop_out_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", pop_out_buttons[x])
                time.sleep(1)
    except  StaleElementReferenceException:
        driver.refresh()
        driver.implicitly_wait(3)
        more_buttons = driver.find_element(By.CLASS_NAME,'show-more-button').click()
        # driver.find_element(By.CLASS_NAME,"nrc6-content-section").click()
        WebDriverWait(driver,3).until(EC.visibility_of_element_located((By.CLASS_NAME,'show-more-button'))).click()
        pop_out_buttons = driver.find_elements(By.CLASS_NAME,"nrc6-content-section")
        if pop_out_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", pop_out_buttons[x])
                time.sleep(1)


In [6]:
# use the ID of the tabs to open the itinary details tab
flight_features_button = driver.find_elements(By.ID,"1b03k8b")
for x in range(len(flight_features_button)):
        try:
                if flight_features_button[x].is_displayed():
                        driver.execute_script("arguments[0].click();", flight_features_button[x])
                time.sleep(0.5)
        except Exception as e :
                print(e)

In [7]:
# open more hidden data so that web scrapping can be done easily
flight_features_button = driver.find_elements(By.CLASS_NAME,"af2q")
for x in range(len(flight_features_button)):
        try:
                if flight_features_button[x].is_displayed():
                        driver.execute_script("arguments[0].click();", flight_features_button[x])
                time.sleep(1)
        except Exception as e :
                print(e)

In [8]:
# use beautiful soup to get all the div for a single flight info
page_source = driver.page_source
soup = BeautifulSoup(page_source,'lxml')
flights = soup.find_all('div', class_='nrc6')



In [9]:
len(flights)

75

In [10]:
for i in range(10):
    print(flights[i])

<div class="nrc6 nrc6-mod-pres-default" data-resultid="432e4a423bc5a8b2b4e6929399ace251"><div class="yuAt yuAt-pres-rounded yuAt-mod-box-shadow" role="group" tabindex="-1"><div class="nrc6-wrapper"><div class="nrc6-inner"><div class="nrc6-content-section"><div class="nrc6-large-header"><div class="btf6"><div class="btf6-container"><div class="btf6-labels"><div class="ZGc- ZGc--mod-margin-left-none ZGc--mod-theme-default ZGc--mod-variant-action ZGc--mod-layout-inline ZGc--mod-padding-default ZGc--mod-size-default ZGc--mod-bold-text ZGc--mod-nowrap">Best</div></div><div class="btf6-actions-wrapper"><div class="AFFP AFFP-body AFFP-s AFFP-res AFFP-emphasis Jav1" role="button" tabindex="0"><div class="Jav1-icon"><span style="transform:translate3d(0,0,0);vertical-align:middle;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale;width:16px;height:16px"><svg class="oLv- oLv--mod-unsaved" fill="inherit" height="100%" style="width:inherit;height:inherit;line-height:inherit;color:

In [12]:
# open a csv file to write the result of the scraped data
csv_file = open('flight_details.csv','w', encoding='utf-8') #open a new file
csv_writer = csv.writer(csv_file) # initializes the CSV writer
csv_writer.writerow(['website','airline','flight_time','num_of_stops','stops', 'total_flight_duration','source_to_dest', 'carry_on_bag',
                     'checked_bag', 'price','flight_class','flght_times',
                     'flight_stations','flight_durations','flight_carriers','flight_layover_duration','flight_layover_locations'
                     ]) #write the header of the csv file


239

In [33]:
flights[0].find('div', class_='VY2U').contents

[<div class="vmXl vmXl-mod-variant-large"><span>6:00 am</span><span class="aOlM"> – </span><span>8:35 am</span></div>,
 <div class="c_cgF c_cgF-mod-variant-default" dir="auto">Ryanair</div>]

In [32]:
flights[0].find('div', class_='VY2U').contents[1].text

'Ryanair'

In [13]:

for flight in flights:
        airline = flight.find('div', class_='VY2U').contents[1].text
        flight_time  = flight.find('div', class_='VY2U').contents[0].text
        no_of_stops = flight.find('div', class_='JWEO').contents[0].text
        stops = flight.find('div', class_='JWEO').contents[1].text
        total_flight_duration = flight.find('div', class_='xdW8').contents[0].text
        source_dest = flight.find('div', class_='xdW8').contents[1].text
        carry_on_bag = flight.find('div', class_='ac27').contents[0].text
        checked_bag = flight.find('div', class_='ac27').contents[1].text
        price = flight.find('div', class_='oVHK').text
        flight_class = flight.find('div', class_='aC3z').text       

        flight_details = flight.find_all('div', class_='g16k')
        times=[]
        stations = []
        for detail in flight_details:
                times.append(detail.find('span', class_='g16k-time').text)
                stations.append(detail.find('span', class_='g16k-station').text)

        # flight_durations = flights.find_all('div', class_='nAz5-duration-text')
        durations = [duration.text for duration in flight.find_all('div', class_='nAz5-duration-text')]
        carriers = [carrier.text for carrier in flight.find_all('div', class_='nAz5-carrier-text')]

        layover_times= []
        layover_locations = []
        layovers = flight.find_all('div', class_='c62AT')
        for layover in layovers:
                layover_times.append(layover.find('div', class_='c62AT-layover-info').contents[0].text)
                layover_locations.append(layover.find('div', class_='c62AT-layover-info').contents[2].text)

        
        csv_writer.writerow([url.split('/')[2],airline,flight_time,no_of_stops,stops,total_flight_duration,source_dest,carry_on_bag,checked_bag,
                                price,flight_class,times,stations,durations,carriers,
                                layover_times,layover_locations])


csv_file.close()


In [17]:
driver.quit()

In [14]:
import pandas as pd

In [15]:
flight_details = pd.read_csv('flight_details.csv')

In [16]:
flight_details.head(15)

,website,airline,flight_time,num_of_stops,stops,total_flight_duration,source_to_dest,carry_on_bag,checked_bag,price,flight_class,flght_times,flight_stations,flight_durations,flight_carriers,flight_layover_duration,flight_layover_locations
0,www.kayak.com,Ryanair,10:00 am – 12:35 pm,nonstop,NaN,2h 35m,FCO-MAD,1,0,$113,Basic,"['10:00 am', '12:35 pm']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 35m'],['Ryanair 9601'],[],[]
1,www.kayak.com,"Ryanair, Air Europa",6:15 am – 1:40 pm,1 stop,ALC,7h 25m,FCO-MAD,1,0,$96,Basic,"['6:15 am', '8:30 am', '12:30 pm', '1:40 pm']","['Rome Fiumicino (FCO)', 'Alicante (ALC)', 'Al...","['2h 15m', '1h 10m']","['Ryanair 4005 · Operated by Malta Air', 'Ai...",['4h 00m'],['Change planes in Alicante (ALC)']
2,www.kayak.com,Ryanair,3:30 pm – 6:05 pm,nonstop,NaN,2h 35m,FCO-MAD,1,0,$124,Basic,"['3:30 pm', '6:05 pm']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 35m'],['Ryanair 9673'],[],[]
3,www.kayak.com,Ryanair,6:00 am – 8:35 am,nonstop,NaN,2h 35m,FCO-MAD,1,0,$132,Basic,"['6:00 am', '8:35 am']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 35m'],['Ryanair 9684 · Operated by Malta Air'],[],[]
4,www.kayak.com,ITA Airways,9:25 pm – 11:59 pm,nonstop,NaN,2h 34m,FCO-MAD,1,0,$120,Economy,"['9:25 pm', '11:59 pm']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 34m'],['ITA Airways 64'],[],[]
5,www.kayak.com,ITA Airways,8:10 am – 10:45 am,nonstop,NaN,2h 35m,FCO-MAD,1,0,$140,Economy,"['8:10 am', '10:45 am']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 35m'],['ITA Airways 58'],[],[]
6,www.kayak.com,ITA Airways,2:40 pm – 5:15 pm,nonstop,NaN,2h 35m,FCO-MAD,1,0,$188,Economy,"['2:40 pm', '5:15 pm']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 35m'],['ITA Airways 60'],[],[]
7,www.kayak.com,Air Europa,8:10 am – 10:45 am,nonstop,NaN,2h 35m,FCO-MAD,1,0,$214,Economy,"['8:10 am', '10:45 am']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 35m'],['Air Europa 3114 · Operated by ITA Airways'],[],[]
8,www.kayak.com,Air Europa,6:25 pm – 9:00 pm,nonstop,NaN,2h 35m,FCO-MAD,1,0,$216,Economy,"['6:25 pm', '9:00 pm']","['Rome Fiumicino (FCO)', 'Madrid Barajas (MAD)']",['2h 35m'],['Air Europa 1048'],[],[]
9,www.kayak.com,"Vueling, Iberia",7:00 am – 11:40 am,1 stop,BCN,4h 40m,FCO-MAD,1,0,$142,Economy,"['7:00 am', '8:55 am', '10:15 am', '11:40 am']","['Rome Fiumicino (FCO)', 'Barcelona-El Prat (B...","['1h 55m', '1h 25m']","['Vueling 6109', 'Iberia 3015']",['1h 20m'],['Change planes in Barcelona (BCN)']


In [ ]:
driver.refresh() # 